In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models

# Constants
IMAGE_SIZE = (128, 128)  # Resize images to 128x128
BATCH_SIZE = 32

# Load labels from the CSV file
def load_labels_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    filenames = df['image_name'].values
    classes = df['tags'].values
    return filenames, classes

# Load images based on filenames from CSV
def load_images_from_folder(folder, filenames):
    images = []
    i =  0
    for filename in filenames:
        # if i > 1000:
        #     break
        if filename.endswith('.jpg'):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, IMAGE_SIZE)
                images.append(img)
                
    return np.array(images)




In [2]:
from sklearn.preprocessing import LabelEncoder

# Load data
filenames, y = load_labels_from_csv('/Users/angel/OneDrive/Desktop/CS 4100/Satellite_Image_Classifier/data/train_v2.csv')
filenames = [name + '.jpg' for name in filenames]
image_folder = '/Users/angel/OneDrive/Desktop/CS 4100/train-jpg/train-jpg/'
X = []

X = load_images_from_folder(image_folder, filenames)
print(np.shape(filenames))

# Normalize the images
X = X.astype('float32') / 255.0

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


print('X: ', len(X), '\nY: ', len(y))

(40479,)
['train_0.jpg', 'train_1.jpg', 'train_2.jpg', 'train_3.jpg', 'train_4.jpg', 'train_5.jpg', 'train_6.jpg', 'train_7.jpg', 'train_8.jpg', 'train_9.jpg']
X:  40479 
Y:  40479


In [5]:
num_classes = len(np.unique(y_encoded))
print(num_classes)

449


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# # Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(np.unique(y)), activation='softmax')  # Use the number of unique classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=7, batch_size=BATCH_SIZE, validation_split=0.2)


predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

accuracy = accuracy_score(predicted_classes, y_test)

print(f"Test accuracy: {accuracy}")

# Example of printing a prediction
for i in range(5):  # Print first 5 predictions
    print(f"Predicted class for test image {i}: {predicted_classes[i]} (Actual class: {y_test[i]})")

c:\Users\angel\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/7
810/810 ━━━━━━━━━━━━━━━━━━━━ 60s 71ms/step - accuracy: 0.3788 - loss: 3.0621 - val_accuracy: 0.4613 - val_loss: 2.4695
Epoch 2/7
810/810 ━━━━━━━━━━━━━━━━━━━━ 56s 69ms/step - accuracy: 0.4744 - loss: 2.3568 - val_accuracy: 0.4785 - val_loss: 2.3787
Epoch 3/7
810/810 ━━━━━━━━━━━━━━━━━━━━ 56s 70ms/step - accuracy: 0.5083 - loss: 2.1680 - val_accuracy: 0.5164 - val_loss: 2.1961
Epoch 4/7
810/810 ━━━━━━━━━━━━━━━━━━━━ 54s 67ms/step - accuracy: 0.5297 - loss: 2.0172 - val_accuracy: 0.5243 - val_loss: 2.1323
Epoch 5/7
810/810 ━━━━━━━━━━━━━━━━━━━━ 54s 67ms/step - accuracy: 0.5472 - loss: 1.8757 - val_accuracy: 0.5319 - val_loss: 2.0702
Epoch 6/7
810/810 ━━━━━━━━━━━━━━━━━━━━ 55s 68ms/step - accuracy: 0.5693 - loss: 1.7525 - val_accuracy: 0.5291 - val_loss: 2.2009
Epoch 7/7
810/810 ━━━━━━━━━━━━━━━━━━━━ 54s 66ms/step - accuracy: 0.5863 - loss: 1.6330 - val_accuracy: 0.5356 - val_loss: 2.1795
253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step
Test accuracy: 0.5453310276679841
Predicted class for t